In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [47]:
d2024 = pd.read_csv("pppub24.csv")
d2023 = pd.read_csv("pppub23.csv")
d2022 = pd.read_csv("pppub22.csv")
d2021 = pd.read_csv("pppub21.csv")

data = pd.concat([d2024, d2023, d2022, d2021], ignore_index=True)

rfe_features100 = data[['HHDFMX', 'CHCARE_YN', 'RINT_YN', 'RETCB_YN' , 'RNT_YN' , 'SS_YN', 'SSI_YN', 'SUR_YN', 'UC_YN', 'VET_YN', 'WC_YN', 'CSP_YN', 'PAW_YN', 'WICYN', 'CHELSEW_YN', 'CHSP_YN', 'POTHVAL', 'RESNSS1', 'RESNSS2', 'RESNSSI1', 'RESNSSI2', 'RETCB_VAL', 'RINT_SC1', 'RINT_SC2', 'RINT_VAL1', 'RINT_VAL2', 'RNT_VAL', 'SRVS_VAL', 'SS_VAL', 'SSI_VAL', 'STRKUC', 'SUBUC', 'SUR_SC1', 'SUR_SC2', 'SUR_VAL1', 'SUR_VAL2', 'TRDINT_VAL', 'TSURVAL1', 'TSURVAL2', 'UC_VAL', 'VET_QVA', 'VET_TYP1', 'VET_TYP2', 'VET_TYP3', 'VET_TYP4', 'VET_TYP5', 'VET_VAL', 'WC_TYPE', 'WC_VAL', 'PAW_MON', 'PAW_TYP', 'PAW_VAL', 'PENINCL', 'PENPLAN', 'CHSP_VAL', 'CSP_VAL', 'ACTC_CRD', 'CTC_CRD', 'DEP_STAT', 'EIT_CRED', 'FEDTAX_AC', 'FEDTAX_BC', 'FICA', 'FILESTAT', 'MARG_TAX', 'PRSWKXPNS', 'STATETAX_A', 'STATETAX_B', 'PERLIS', 'POV_UNIV', 'COV', 'COV_CYR', 'COV_MULT_CYR', 'NOCOV_CYR', 'NOW_COV', 'NOW_PUB', 'PUB', 'PUB_CYR', 'NOW_DEPPRIV', 'NOW_OUTPRIV', 'NOW_OWNPRIV', 'NOW_PRIV', 'OUTPRIV', 'OWNPRIV', 'PRIV', 'PRIV_CYR', 'DEPGRP', 'GRPFTYP', 'GRPFTYP2', 'MRK', 'NOW_CAID', 'NOW_PCHIP', 'NOW_MCARE', 'NOW_IHSFLG', 'NOW_VACARE', 'PECOULD', 'ESIOFFER']]
rfe_features50 = data[['CHCARE_YN', 'CHELSEW_YN', 'CHSP_VAL', 'CHSP_YN', 'CSP_VAL', 'CSP_YN', 'ACTC_CRD', 'CTC_CRD', 'DEP_STAT', 'EIT_CRED', 'FEDTAX_AC', 'FEDTAX_BC', 'FICA', 'FILESTAT', 'MARG_TAX', 'PRSWKXPNS', 'STATETAX_A', 'STATETAX_B', 'PERLIS', 'POV_UNIV', 'COV', 'COV_CYR', 'COV_MULT_CYR', 'NOCOV_CYR', 'NOW_COV', 'NOW_PUB', 'PUB', 'DEPPRIV', 'NOW_DEPPRIV', 'NOW_OUTPRIV', 'NOW_OWNPRIV', 'NOW_PRIV', 'OUTPRIV', 'OWNPRIV', 'PRIV', 'PRIV_CYR', 'GRP', 'GRPFTYP', 'GRPFTYP2', 'NOW_MCAID', 'NOW_CAID', 'NOW_PCHIP', 'NOW_MCARE', 'NOW_IHSFLG', 'ESICOULD']]
rfe_features20 = data[['CSP_VAL', 'CSP_YN', 'FEDTAX_AC', 'FEDTAX_BC', 'FICA', 'FILESTAT', 'PRSWKXPNS', 'STATETAX_A', 'STATETAX_B', 'PERLIS', 'COV_MULT_CYR', 'DEPPRIV', 'NOW_OWNPRIV', 'OWNPRIV', 'GRPFTYP2', 'NOW_MCARE']]

In [48]:
X = rfe_features100
y = data['PTOT_R']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

#0.6871133284249623 - 3
#0.6947945626971133 - 20
#0.7049482972714586 - 50
#0.9748189580334282 - 100
#0.9881924374457424 - 294

Accuracy: 0.97493983582598


In [106]:
from sklearn.tree import _tree

#change target feature class
#index 2 - class 1
#index 26 - class 2
#index 16 - class 3
#index 40 - class 40
#index 10 - class 41
first_class = y_train.unique()[10]

#change row
index = y_train[y_train == first_class].index[2]
instance = X_train.loc[index].values.reshape(1, -1)






node_indicator = clf.decision_path(instance)
leaf_id = clf.apply(instance)[0]

tree_ = clf.tree_
feature_names = [X.columns[i] if i != _tree.TREE_UNDEFINED else "undefined!" for i in tree_.feature]

print(f"Decision path for a sample from class {first_class}:\n")

for node_id in node_indicator.indices:
    if leaf_id == node_id:
        print(f"==> Reached leaf node {node_id}")
        break

    feature = feature_names[node_id]
    threshold = tree_.threshold[node_id]
    if instance[0, tree_.feature[node_id]] <= threshold:
        condition = "<="
    else:
        condition = ">"

    value = instance[0, tree_.feature[node_id]]
    print(f"Node {node_id}: {feature} (value: {value:.2f}) {condition} {threshold:.2f}")

Decision path for a sample from class 41:

Node 0: VET_YN (value: 2.00) > 0.50
Node 2: FICA (value: 7650.00) > 7631.50
Node 16266: DEPGRP (value: 1.00) > 0.50
Node 17064: FEDTAX_BC (value: 0.00) <= 13772.00
Node 17065: MARG_TAX (value: 0.00) <= 5.00
Node 17066: RETCB_VAL (value: 23500.00) > 3.00
Node 17392: FICA (value: 7650.00) > 7641.50
Node 17394: POTHVAL (value: 6700.00) > -9768.50
Node 17398: RINT_SC1 (value: 1.00) <= 2.50
Node 17399: POTHVAL (value: 6700.00) > 1.50
Node 17419: FICA (value: 7650.00) <= 9902.50
Node 17420: FICA (value: 7650.00) <= 9885.50
Node 17421: RNT_VAL (value: 0.00) <= 0.50
Node 17422: RETCB_VAL (value: 23500.00) <= 29750.00
Node 17423: NOCOV_CYR (value: 1.00) <= 1.50
Node 17424: RETCB_VAL (value: 23500.00) > 2008.00
Node 17434: RINT_VAL2 (value: 1000.00) > 91.50
Node 17450: RINT_VAL2 (value: 1000.00) > 96.50
Node 17452: RETCB_VAL (value: 23500.00) > 6050.00
==> Reached leaf node 17468


c:\Users\aa398\Desktop\MLProject\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\aa398\Desktop\MLProject\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
